In [1]:
from keras.models import Sequential, Model
from keras.layers import Input, Dense, Conv2D, Dropout, Flatten, Conv2DTranspose, BatchNormalization, LeakyReLU, Reshape
from keras.datasets import fashion_mnist
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

In [ ]:
# Load and preprocess the Fashion MNIST dataset
(train_x, train_y), (val_x, val_y) = fashion_mnist.load_data()
train_x = train_x / 255.0
val_x = val_x / 255.0
train_x = train_x.reshape(-1, 28, 28, 1)

In [ ]:
# Display some samples of the dataset
fig, axe = plt.subplots(2, 2)
idx = 0
for i in range(2):
    for j in range(2):
        axe[i, j].imshow(train_x[idx].reshape(28, 28), cmap='gray')
        idx += 1

# Normalize data to range [-1, 1]
train_x = train_x * 2 - 1
print(train_x.max(), train_x.min())

In [ ]:
# Define the generator model
generator = Sequential([
    Dense(512, input_shape=[100]),
    LeakyReLU(alpha=0.2),
    BatchNormalization(momentum=0.8),
    Dense(256),
    LeakyReLU(alpha=0.2),
    BatchNormalization(momentum=0.8),
    Dense(128),
    LeakyReLU(alpha=0.2),
    BatchNormalization(momentum=0.8),
    Dense(784),
    Reshape([28, 28, 1])
])
generator.summary()

In [ ]:
# Define the discriminator model
discriminator = Sequential([
    Dense(1, input_shape=[28, 28, 1]),
    Flatten(),
    Dense(256),
    LeakyReLU(alpha=0.2),
    Dropout(0.5),
    Dense(128),
    LeakyReLU(alpha=0.2),
    Dropout(0.5),
    Dense(64),
    LeakyReLU(alpha=0.2),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])
discriminator.summary()

In [ ]:
# Combine generator and discriminator into a GAN model
GAN = Sequential([generator, discriminator])

In [ ]:
# Compile the discriminator model
discriminator.compile(optimizer='adam', loss='binary_crossentropy')

In [ ]:
# Set discriminator as non-trainable before compiling GAN
discriminator.trainable = False

In [ ]:
# Compile the GAN model
GAN.compile(optimizer='adam', loss='binary_crossentropy')
GAN.summary()

In [ ]:
epochs = 30
batch_size = 100
noise_shape = 100

# Training loop
with tf.device('/gpu:0'):
    for epoch in range(epochs):
        print(f"Currently on Epoch {epoch + 1}")
        for i in range(train_x.shape[0] // batch_size):
            if (i + 1) % 100 == 0:
                print(f"\tCurrently on batch number {i + 1} of {train_x.shape[0] // batch_size}")
            noise = np.random.normal(size=[batch_size, noise_shape])
            gen_image = generator.predict_on_batch(noise)
            train_dataset = train_x[i * batch_size: (i + 1) * batch_size]
            train_label = np.ones(shape=(batch_size, 1))
            d_loss_real = discriminator.train_on_batch(train_dataset, train_label)
            train_label = np.zeros(shape=(batch_size, 1))
            d_loss_fake = discriminator.train_on_batch(gen_image, train_label)
            noise = np.random.normal(size=[batch_size, noise_shape])
            train_label = np.ones(shape=(batch_size, 1))
            d_g_loss_batch = GAN.train_on_batch(noise, train_label)
        if epoch % 10 == 0:
            samples = 10
            x_fake = generator.predict(np.random.normal(loc=0, scale=1, size=(samples, 100)))
            for k in range(samples):
                plt.subplot(2, 5, k + 1)
                plt.imshow(x_fake[k].reshape(28, 28), cmap='gray')
                plt.xticks([])
                plt.yticks([])
            plt.tight_layout()
            plt.show()

print('Training is complete')

In [ ]:
# Generate images from noise
noise = np.random.normal(size=[10, noise_shape])
gen_image = generator.predict(noise)

In [ ]:
# Display generated images
fig, axe = plt.subplots(2, 5)
fig.suptitle('Generated Images from Noise using GANs')
idx = 0
for i in range(2):
    for j in range(5):
        axe[i, j].imshow(gen_image[idx].reshape(28, 28), cmap='gray')
        idx += 1
plt.show()

In [4]:
import tensorflow as tf
print(tf.keras.utils.get_file('', ''))


ValueError: Can't parse the file name from the origin provided: ''.Please specify the `fname` as the input param.